![](https://wherobots.com/wp-content/uploads/2023/12/Inline-Blue_Black_onWhite@3x.png)

# Havasu Raster ETL example

In this example we demonstrate:

* working with the EuroSAT raster dataset as Havasu tables
* raster opertions 
* handling CRS transforms, and 
* benchmarking raster geometry operations


Read more about [Havasu](https://docs.wherobots.com/latest/references/havasu/introduction/), and [WherobotsDB Raster support](https://docs.wherobots.com/latest/references/havasu/raster/raster-overview/) in the documentation.

# Launch Spark Job

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
from sedona.spark import *
import geopandas as gpd

# Define sedona context

In [ ]:
config = SedonaContext.builder().appName('havasu-iceberg-raster-etl')\
    .getOrCreate()
sedona = SedonaContext.create(config)

# Load Raster Datasets

## EuroSAT

In [ ]:
eurosat_path = 's3://wherobots-examples/data/eurosat_small'
df_binary = sedona.read.format("binaryFile").option("pathGlobFilter", "*.tif").option("recursiveFileLookup", "true").load(eurosat_path)
df_geotiff = df_binary.withColumn("rast", expr("RS_FromGeoTiff(content)")).withColumn("name", expr("reverse(split(path, '/'))[0]")).select("name", "length", "rast")
df_geotiff.show(5, False)

# Save Raster Datasets to Havasu

In [ ]:
sedona.sql("CREATE NAMESPACE IF NOT EXISTS wherobots.test_db")
sedona.sql("DROP TABLE IF EXISTS wherobots.test_db.eurosat_ms")
df_geotiff.coalesce(16).writeTo("wherobots.test_db.eurosat_ms").create()

## Save another copy of EuroSAT partitioned by SRID

In [ ]:
sedona.sql("DROP TABLE IF EXISTS wherobots.test_db.eurosat_ms_srid")
df_rast_havasu = sedona.table("wherobots.test_db.eurosat_ms")
df_rast_havasu.withColumn("srid", expr("RS_SRID(rast) as srid"))\
    .sort('srid')\
    .write.format("havasu.iceberg").partitionBy("srid")\
    .saveAsTable("wherobots.test_db.eurosat_ms_srid")

## Reload Havasu Rasters

In [ ]:
df_rast_havasu = sedona.table("wherobots.test_db.eurosat_ms")
df_rast_havasu_srid = sedona.table('wherobots.test_db.eurosat_ms_srid')

# Test Raster Functions

## Test Basic Raster Property Accessors

In [ ]:
df_rast_havasu.selectExpr("name", "RS_Envelope(rast) as env", "RS_Metadata(rast) as meta").show(5)

## Test Pixel Data Accessors

In [ ]:
df_rast_havasu.selectExpr("name", "RS_Value(rast, ST_Centroid(RS_Envelope(rast))) as centroid_val").show(5, False)

## Test Band Accessors

In [ ]:
df_rast_havasu.selectExpr("name", "RS_BandAsArray(rast, 1) as band1", "RS_BandAsArray(rast, 2) as band2", "RS_BandAsArray(rast, 3) as band3").show(5)

## Test Preprocessing for DeepSatV2

In [ ]:
df_extra_features = df_rast_havasu\
    .withColumn("band_red", expr("RS_BandAsArray(rast, 4)"))\
    .withColumn("band_green", expr("RS_BandAsArray(rast, 3)"))\
    .withColumn("band_nir", expr("RS_BandAsArray(rast, 8)"))\
    .withColumn("band_swir1", expr("RS_BandAsArray(rast, 12)"))\
    .withColumn("band_swir2", expr("RS_BandAsArray(rast, 13)"))\
    .withColumn("band_ndwi", expr("RS_NormalizedDifference(band_green, band_nir)"))\
    .withColumn("band_mndwi", expr("RS_NormalizedDifference(band_green, band_swir1)"))\
    .withColumn("band_ndmi", expr("RS_NormalizedDifference(band_nir, band_swir1)"))\
    .withColumn("band_ndvi", expr("RS_NormalizedDifference(band_nir, band_red)"))\
    .withColumn("band_awei", expr("RS_Subtract(RS_MultiplyFactor(RS_Subtract(band_green, band_swir1), 4), RS_Add(RS_MultiplyFactor(band_nir, 0.25), RS_MultiplyFactor(band_swir2, 2.75)))"))\
    .withColumn("band_builtup", expr("RS_NormalizedDifference(band_swir1, band_nir)"))\
    .withColumn("band_rvi", expr("RS_Divide(band_nir, RS_LogicalOver(band_red, RS_Array(array_size(band_red), 1e-12)))"))\
    .selectExpr("name", "RS_Mean(band_ndwi) as mean_ndwi", "RS_Mean(band_mndwi) as mean_mndwi", "RS_Mean(band_ndmi) as mean_ndmi", "RS_Mean(band_ndvi) as mean_ndvi", "RS_Mean(band_awei) as mean_awei", "RS_Mean(band_builtup) as mean_builtup", "RS_Mean(band_rvi) as mean_rvi")
df_extra_features.show(5)

In [ ]:
df_extra_features = df_rast_havasu\
    .withColumn("ndvi", expr("RS_MapAlgebra(rast, 'd', 'out = (rast[3] - rast[7]) / (rast[3] + rast[7]);', null)"))\
    .withColumn("awei", expr("RS_MapAlgebra(rast, 'd', 'out = (0.25 * rast[7] + 2.75 * rast[12]) - 4 * (rast[11] - rast[2]);', null)"))\
    .withColumn("rvi", expr("RS_MapAlgebra(rast, 'd', 'out = rast[7] / max(rast[3], 0.000001);', null)"))\
    .withColumn("mean_ndvi", expr("RS_Mean(RS_BandAsArray(ndvi, 1))"))\
    .withColumn("mean_awei", expr("RS_Mean(RS_BandAsArray(awei, 1))"))\
    .withColumn("mean_rvi", expr("RS_Mean(RS_BandAsArray(rvi, 1))"))
df_extra_features.show(5)

In [ ]:
df_extra_features.where("mean_awei > 0.5").count()

# Data Visualization

We'll visualize the bounding boxes of the rasters in EuroSAT. Here we can see the importance of handling the CRS of rasters properly.

* `df_rast_env` contains envelopes of rasters in EuroSAT
* `df_rast_env_srid` contains envelopes of rasters in EuroSAT transformed to CRS:4326 (EPSG:4326 in lon-lat axis order)

In [ ]:
df_rast_env = df_rast_havasu.selectExpr('name', "RS_Envelope(rast) as env", "RS_SRID(rast) as srid")

df_rast_env_4326 = df_rast_havasu.selectExpr('name', "RS_Envelope(rast) as env", "RS_SRID(rast) as srid")\
    .withColumn("env_4326", expr("ST_Transform(env, concat('epsg:', srid), 'epsg:4326')"))\
    .select("name", "env_4326", "srid")

Now let's plot the datasets. We plot the transformed envelopes of the rasters, and color the geometries by the original SRID of the rasters. We know that the CRS of the rasters are in UTM, so they appears to be grouped by vertical stripes.

In [ ]:
rasterMap_4326 = SedonaKepler.create_map()
SedonaKepler.add_df(rasterMap_4326, df_rast_env_4326, name="raster-bounds")
rasterMap_4326

You can see that the rasters in `df_rast_env` are not plotted correctly because the rasters are in various UTM CRS, and it is not meaningful to plot rasters in different CRS together.

The reason why plotting the envelopes of rasters without considering their CRS results in a long stripe is that the CRS of the rasters are in UTM, and rasters in different UTM zones have the same coordinate range since they share the same false easting and northing.

In [ ]:
gdf_rast_env = df_rast_env.toPandas()
gdf_rast_env = gpd.GeoDataFrame(gdf_rast_env, geometry='env')
gdf_rast_env['boundary'] = gdf_rast_env.boundary
gdf_rast_env.set_geometry('boundary', inplace=True)
gdf_rast_env.plot(column='srid')

# Use a better partitioner for faster range query

Now let's use the H3 cell ID of the centroid of the raster in EPSG:4326 to partition the dataset. This will result in a better partitioning scheme for range query.

In [ ]:
df_rast_havasu_h3 = df_rast_havasu\
    .withColumn("centroid", expr("ST_Transform(ST_Centroid(RS_Envelope(rast)), concat('epsg:', RS_SRID(rast)), 'epsg:4326')"))\
    .withColumn("h3_cell_id", expr("array_max(ST_H3CellIDs(centroid, 1, false))"))

If we plot the centroid of rasters using different colors for different H3 cell IDs, we can see that the rasters are partitioned into different H3 cells.

In [ ]:
gdf_rast_havasu_h3 = gpd.GeoDataFrame(df_rast_havasu_h3.select("centroid", "h3_cell_id").toPandas(), geometry='centroid', crs='EPSG:4326')
gdf_rast_havasu_h3.plot(column='h3_cell_id', cmap='flag', markersize=1)

Now let's save the dataset partitioned by H3 cell ID and reload it.

In [ ]:
sedona.sql("DROP TABLE IF EXISTS wherobots.test_db.eurosat_ms_h3")
df_rast_havasu_h3.sort("h3_cell_id").write.format("havasu.iceberg").partitionBy("h3_cell_id").saveAsTable("wherobots.test_db.eurosat_ms_h3")

In [ ]:
df_rast_havasu_h3 = sedona.table('wherobots.test_db.eurosat_ms_h3')

# Benchmarking Raster Functions

This is just a simple benchmark running on EuroSAT dataset. It could give the user a rough idea of how havasu in-db raster performs compared to GeoTiff when processing lots of tiny raster images.

In [ ]:
import time

def benchmark_query(df_dict, bench_func, num_runs=1):
    cost_dict = {}
    for name, df in df_dict.items():
        print(f"Running benchmark for {name}")
        cost_dict[name] = []
        for i in range(1, num_runs + 1):
            print(f"Run #{i} for {name}")
            start = time.time()
            result = bench_func(df)
            end = time.time()
            cost = end - start
            print(f"Run #{i} for {name} took {cost} seconds, result: {result}")
            cost_dict[name].append(cost)
    # print summary
    for name, costs in cost_dict.items():
        print(f"Summary for {name} - runs: {len(costs)}, mean: {sum(costs)/len(costs)}, min: {min(costs)}, max: {max(costs)}")
    return cost_dict

In [ ]:
df_dict = {
    'havasu': df_rast_havasu,
    'havasu_srid': df_rast_havasu_srid,
    'havasu_h3': df_rast_havasu_h3,
    'geotiff': df_geotiff
}

## Scanning the entire dataset and extract basic raster properties

In [ ]:
def bench_func(df):
    return df.withColumn("num_bands", expr("RS_NumBands(rast)")).where("num_bands IS NOT NULL").count()

benchmark_query(df_dict, bench_func, num_runs=1)

## Scanning the entire dataset and extract bands

In [ ]:
def bench_func(df):
    return df.selectExpr("name", "RS_BandAsArray(rast, 1) as band1", "RS_BandAsArray(rast, 2) as band2")\
        .selectExpr("RS_NormalizedDifference(band1, band2) as band_nd")\
        .where("array_size(band_nd) > 0")\
        .count()

benchmark_query(df_dict, bench_func, num_runs=1)

## Scanning the entire dataset and extract pixel values

In [ ]:
def bench_func(df):
    return df.selectExpr("RS_Value(rast, ST_Centroid(RS_Envelope(rast))) as centroid_val")\
        .where("centroid_val IS NOT NULL")\
        .count()
    
benchmark_query(df_dict, bench_func, num_runs=1)

## Range query

We run several range queries on the EuroSAT dataset using the following query windows. The query windows were specified as rectangles in EPSG:4326.

In [ ]:
query_windows = {
    'spain_madrid': 'ST_SetSRID(ST_PolygonFromEnvelope(-4.7803,39.5882, -2.7782,40.9276), 4326)',
    'cesko_praha': 'ST_SetSRID(ST_PolygonFromEnvelope(13.2747,49.2297, 16.3189,51.0516), 4326)',
    'france_paris': 'ST_SetSRID(ST_PolygonFromEnvelope(1.299,48.156, 3.566,49.575), 4326)',
}

In [ ]:
def bench_query_func(df, qw_expr):
    return df.where(f"RS_Intersects(rast, {qw_expr})").count()

for qw_name, qw_exr in query_windows.items():
    print(f"Running benchmark using query window {qw_name}")
    benchmark_query(df_dict, lambda df: bench_query_func(df, qw_exr), num_runs=1)